In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


# Lecture 01: Single Agent

This notebook demonstrates a basic financial assistant agent that uses web search to answer financial questions.
The agent "collaborates" with itself using the ReAct pattern.

## How the Agent Works: ReAct Pattern

The agent follows the **ReAct** (Reasoning + Acting) pattern, which combines reasoning and acting in an iterative loop:

```mermaid
graph TD
    Start([👤 User Question 👤]) --> Reason[🤖 Agent: Analyze & Reason<br/>- Understand question<br/>- Plan approach<br/>- Decide on tools]
    Reason --> Decision{Need<br/>Tools?}
    
    Decision -->|No| Response[📝 Generate<br/>Final Response]
    Decision -->|Yes| CallTool[🔧 Call Tool<br/>Web Search]
    
    CallTool --> Results[📊 Receive<br/>Tool Results]
    Results --> React[🤖 Agent: React<br/>- Process results<br/>- Integrate information<br/>- Decide next step]
    
    React --> Decision
    
    Response --> End([✅ Return Answer&nbsp;<br/>to User])
    
    style Start fill:#e1f5ff,stroke:#01579b,stroke-width:3px
    style Reason fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Decision fill:#f3e5f5,stroke:#4a148c,stroke-width:2px
    style CallTool fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style Results fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style React fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Response fill:#e0f2f1,stroke:#004d40,stroke-width:2px
    style End fill:#e1f5ff,stroke:#01579b,stroke-width:3px
```

**Key Points:**
- **Reasoning**: The agent analyzes the question and decides what actions to take
- **Acting**: The agent calls tools (like web search) to gather information
- **Iteration**: This process repeats until the agent has enough information (shown by the loop back from React to Decision)
- **Reflection**: After each tool call, the agent reflects on the results and decides next steps


In [ ]:
# Setup: Import necessary modules
from IPython.display import Markdown, display
from pydantic_ai import Agent
from common.tools import web_search_tool
from common.utils import (
    create_agent_model,
    setup_logging,
    setup_logfire,
    get_current_date,
)

# Initialize logging
setup_logging()

# Initialize Logfire instrumentation (for observability)
setup_logfire(
    service_name="financial-assistant-lecture01",
    start_message="🚀 Lecture 01 - Financial Assistant Notebook Started",
)

In [ ]:
# Create the financial assistant agent
financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool],
    system_prompt=f"""
You are a knowledgeable financial assistant. Your role is to help users with financial questions,
investment advice, market analysis, and general financial guidance.
IMPORTANT CONTEXT:
- Today's date is {get_current_date()}
- Always provide information in the context of this current date


When answering questions:
1. Use the web_search tool to get current, up-to-date financial information when relevant
2. Combine web search results with your financial knowledge
3. Provide clear, actionable advice
4. Always mention when information is based on web search vs. general knowledge
5. Include appropriate disclaimers for investment advice
6. Be helpful but remind users to consult with financial professionals for major decisions

Keep your responses informative but concise, and always prioritize accuracy and user safety.
""",
)

In [ ]:
# Ask a question
question = "What is the current price of Bitcoin?"
result = await financial_agent.run(question)

# Display with nice Markdown formatting
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
# Escape dollar signs to prevent LaTeX interpretation
display(Markdown(result.output.replace("$", "\\$")))

**Question:** What is the current price of Bitcoin?

---

Based on the most recent web search results (as of today):

- The current price of Bitcoin (BTC) is approximately $90,188 to $90,240 USD.

Please note that cryptocurrency prices are highly volatile and can change within minutes. For the most accurate and updated price, you can check reputable platforms like CoinMarketCap, CoinGecko, or Coinbase.

If you plan to trade or invest, always verify prices directly on your chosen exchange and consult with a financial professional for significant decisions.

In [ ]:
# Try another question
question = "Find 3 trending stocks and for each one provide an investor's report"
result = await financial_agent.run(question)

# Display with nice Markdown formatting
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
# Escape dollar signs to prevent LaTeX interpretation
display(Markdown(result.output.replace("$", "\\$")))

**Question:** Find 3 trending stocks and for each one provide an investor's report

---

Based on the latest market news (as of today), here are three trending stocks:

1. **Nvidia (NVDA)**
2. **Tesla (TSLA)**
3. **Micron Technology (MU)**

Below are brief investor reports for each. This information is based on current news and analyst insights. For major investment decisions, please consult with a qualified financial advisor.

---

### 1. Nvidia (NVDA)

**Recent Performance & Sentiment:**  
Nvidia remains one of the most actively traded and closely watched stocks. Analysts continue to rate it as “outperform,” citing strong momentum in AI chip demand. Recent news confirms positive developments, including the US government allowing Nvidia’s cutting-edge H200 AI chips to be supplied to select Chinese customers, which could open new growth channels.

**Key Drivers:**
- Leadership in AI and data center semiconductors
- Expanding markets in gaming, automotive, and cloud computing
- New regulatory permissions boosting global reach

**Risk Factors:**
- Valuation is historically high, pricing in substantial growth
- Global chip competition and geopolitical risks

**Summary:**  
Nvidia is favored by analysts for its technological leadership and growth prospects, especially in AI. Due diligence is advised due to price volatility and sector competition.

---

### 2. Tesla (TSLA)

**Recent Performance & Sentiment:**  
Tesla continues to trend as a top idea for 2026, recommended by Deutsche Bank and appearing high on analysts’ watchlists. Recent pullbacks reflect market-wide volatility, but the company’s ongoing innovation in EVs and energy solutions keeps investor interest alive.

**Key Drivers:**
- Global leadership in electric vehicles
- Diversification into energy storage and solar
- New model launches and supply chain improvements

**Risk Factors:**
- High dependence on consumer demand and competition from other automakers
- Regulatory risk in key markets (US, China, Europe)

**Summary:**  
Tesla remains a growth story with ongoing innovations. Consideration of volatility and sector risks is important before investing.

---

### 3. Micron Technology (MU)

**Recent Performance & Sentiment:**  
Micron is gaining attention as HSBC upgraded the stock to “Buy”, citing expected leadership in memory technology and the start of a new upcycle for DRAM/NAND markets. The stock has recently hit new highs.

**Key Drivers:**
- Leadership in DRAM and NAND flash memory technologies
- Benefiting from renewed demand in AI, cloud, and mobile
- Positive cycle outlook by major analysts

**Risk Factors:**
- Exposure to cyclical swings in semiconductor demand
- Supply chain and pricing pressures

**Summary:**  
Micron is considered well-positioned for growth as industry tailwinds pick up. Investors should keep watch for cyclical risks.

---

**Disclaimer:**  
This report is based on recent news and analyst outlooks, not deep fundamental analysis. Investing in stocks carries risks, including potential loss of principal. Always consult a financial professional before making substantial investment decisions. If you'd like a more detailed analysis of any of these stocks, just let me know!